In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json!/.kaggle/

cp: missing destination file operand after 'kaggle.json!/.kaggle/'
Try 'cp --help' for more information.


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 98% 1.04G/1.06G [00:09<00:00, 225MB/s]
100% 1.06G/1.06G [00:09<00:00, 122MB/s]


In [ ]:
import zipfile
z = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
z.extractall('/content')
z.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

Using Generator hlep to divide data in batches to process large amount of data as this kind of data we are working on is very large we need generator for both training and validation.


In [ ]:
# generator image datasets from directory
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred', # it label cat and dog 1 and 0 respectively or viceversa
    batch_size = 32,
    image_size = (256,256) # we need this as size of each image in data are not same



)

Found 20000 files belonging to 2 classes.


In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    batch_size = 32,
    image_size = (256,256) # we need this as size of each image in data are not same



)

Found 5000 files belonging to 2 classes.


In [ ]:
# Normalise all pixels  1-255 into 0-1
def process(image,label):
  image=tf.cast(image/255.,tf.float32)
  return image,label
train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size = (3,3),padding ='valid',activation = 'relu',input_shape = (256,256,3)))
model.add(MaxPooling2D(pool_size = (2,2),strides = 2,padding = 'valid'))

model.add(Conv2D(64,kernel_size = (3,3),padding ='valid',activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2),strides = 2,padding = 'valid'))

model.add(Conv2D(128,kernel_size = (3,3),padding ='valid',activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2),strides = 2,padding = 'valid'))
model.add(Flatten())

model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [ ]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [ ]:
history = model.fit(train_ds,epochs = 10,validation_data = test_ds)

Epoch 1/10
625/625 [==============================] - 3167s 5s/step - loss: 0.6100 - accuracy: 0.6528 - val_loss: 0.4940 - val_accuracy: 0.7598
Epoch 2/10
 85/625 [===>..........................] - ETA: 41:31 - loss: 0.5080 - accuracy: 0.7507